In [60]:
# Importing dependecies:
import numpy as np
import pandas as pd
import pickle
from collections import defaultdict
import re
from bs4 import BeautifulSoup
import sys
import os
from collections import Counter
import tqdm

from nltk.tokenize import sent_tokenize

import keras
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import keras.preprocessing.text as kpt
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import text
from keras.preprocessing.sequence import pad_sequences
from keras.utils.vis_utils import plot_model
from keras.models import Model, Sequential
from keras.layers import Input, Embedding, Dense, Flatten, Dropout, Activation
from keras.layers.convolutional import Conv1D, MaxPooling1D
from keras.layers.merge import concatenate

import matplotlib.pyplot as plt
import seaborn as sns
plt.switch_backend('agg')
%matplotlib inline
from pandas import compat
compat.PY3 = True

# Configuring Notebook environment:
sns.set()
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

plt.rcParams['figure.figsize'] = (10.0, 7.5)
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

In [2]:
df = pd.read_csv('data/strings/df_clean.csv', index_col=0)
df.head()

,title,ingredients,instructions,ingredients_vector,instructions_vector
p3pKOD6jIHEcjf20CCXohP8uqkG5dGi,grammie hamblets deviled crab,celery finely chopped green pepper finely chop...,toss ingredients lightly spoon buttered baking...,"['celery', 'finely', 'chopped', 'green', 'pepp...","['toss', 'ingredients', 'lightly', 'spoon', 'b..."
S7aeOIrsrgT0jLP32jKGg4j.o9zi2DO,infineon raceway baked beans,skirt steak cut inch dicekosher salt fresh cra...,watch make recipe sprinkle steak salt pepper s...,"['skirt', 'steak', 'cut', 'inch', 'dicekosher'...","['watch', 'make', 'recipe', 'sprinkle', 'steak..."
o9MItV9txfoPsUQ4v8b0vh1.VdjwfsK,southwestern black bean dip,cups dried black beans picked rinsed cups wate...,saucepan let beans soak enough cold water cove...,"['cups', 'dried', 'black', 'beans', 'picked', ...","['saucepan', 'let', 'beans', 'soak', 'enough',..."
5l1yTSYFifF/M2dfbD6DX28WWQpLWNK,sour cream noodle bake,ground chuckone tomato sauce saltfreshly groun...,watch make recipe preheat oven degrees f brown...,"['ground', 'chuckone', 'tomato', 'sauce', 'sal...","['watch', 'make', 'recipe', 'preheat', 'oven',..."
kRBQSWtqYWqtkb34FGeenBSbC32gIdO,sushi renovation,rice brown mediumgrain cookedcup quinoacup swe...,special equipment sushi mat cook brown rice qu...,"['rice', 'brown', 'mediumgrain', 'cookedcup', ...","['special', 'equipment', 'sushi', 'mat', 'cook..."


## Tokenizing Titles:

In [3]:
df['title_tokenized'] = list(df['title'].apply(sent_tokenize).astype(str))
df.head()

,title,ingredients,instructions,ingredients_vector,instructions_vector,title_tokenized
p3pKOD6jIHEcjf20CCXohP8uqkG5dGi,grammie hamblets deviled crab,celery finely chopped green pepper finely chop...,toss ingredients lightly spoon buttered baking...,"['celery', 'finely', 'chopped', 'green', 'pepp...","['toss', 'ingredients', 'lightly', 'spoon', 'b...",['grammie hamblets deviled crab']
S7aeOIrsrgT0jLP32jKGg4j.o9zi2DO,infineon raceway baked beans,skirt steak cut inch dicekosher salt fresh cra...,watch make recipe sprinkle steak salt pepper s...,"['skirt', 'steak', 'cut', 'inch', 'dicekosher'...","['watch', 'make', 'recipe', 'sprinkle', 'steak...",['infineon raceway baked beans']
o9MItV9txfoPsUQ4v8b0vh1.VdjwfsK,southwestern black bean dip,cups dried black beans picked rinsed cups wate...,saucepan let beans soak enough cold water cove...,"['cups', 'dried', 'black', 'beans', 'picked', ...","['saucepan', 'let', 'beans', 'soak', 'enough',...",['southwestern black bean dip']
5l1yTSYFifF/M2dfbD6DX28WWQpLWNK,sour cream noodle bake,ground chuckone tomato sauce saltfreshly groun...,watch make recipe preheat oven degrees f brown...,"['ground', 'chuckone', 'tomato', 'sauce', 'sal...","['watch', 'make', 'recipe', 'preheat', 'oven',...",['sour cream noodle bake']
kRBQSWtqYWqtkb34FGeenBSbC32gIdO,sushi renovation,rice brown mediumgrain cookedcup quinoacup swe...,special equipment sushi mat cook brown rice qu...,"['rice', 'brown', 'mediumgrain', 'cookedcup', ...","['special', 'equipment', 'sushi', 'mat', 'cook...",['sushi renovation']


## Defining and Splitting Data:

In [27]:
X = df['ingredients_vector']
y = df['title_tokenized']
X

p3pKOD6jIHEcjf20CCXohP8uqkG5dGi    ['celery', 'finely', 'chopped', 'green', 'pepp...
S7aeOIrsrgT0jLP32jKGg4j.o9zi2DO    ['skirt', 'steak', 'cut', 'inch', 'dicekosher'...
o9MItV9txfoPsUQ4v8b0vh1.VdjwfsK    ['cups', 'dried', 'black', 'beans', 'picked', ...
5l1yTSYFifF/M2dfbD6DX28WWQpLWNK    ['ground', 'chuckone', 'tomato', 'sauce', 'sal...
kRBQSWtqYWqtkb34FGeenBSbC32gIdO    ['rice', 'brown', 'mediumgrain', 'cookedcup', ...
                                                         ...                        
4bfMWxlbKhx/McJq/89k0SBdw.VvAzW    ['ears', 'fresh', 'corn', 'heads', 'belgian', ...
T8lWBA1fcVdjxhMSWuoAbGoy5Lj.A8m    ['plum', 'tomatoessalt', 'sugar', 'zucchini', ...
f/coffo2TMs2J2gq5nTOUIqH2TRAkui    ['tablespoons', 'olive', 'oil', 'tablespoons',...
q3aDJc4zoEF5QT4e7Mn.ieQwV.DyHwS    ['ounces', 'butter', 'ounces', 'bittersweet', ...
7cXA77UpdDtIfBug2v6lEVIuV3Zcvhm    ['cans', 'restaurantstyle', 'condensed', 'crab...
Name: ingredients_vector, Length: 59612, dtype: object

In [46]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [47]:
print(len(X_train), 'train sequences')
print(len(X_test), 'test sequences')

47689 train sequences
11923 test sequences


In [48]:
num_classes = len(y_train) + 1
num_classes

47690

## Parameters:

In [49]:
max_words = 1000
batch_size = 32
epochs = 5

In [50]:
print('Vectorizing sequence data...')
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X_train)
# X_train = tokenizer.texts_to_sequences(X_train)
# X_test = tokenizer.fit_on_texts(X_test)
# print('x_train shape:', X_train.shape)
# print('x_test shape:', X_test.shape)

Vectorizing sequence data...


In [54]:
dictionary = tokenizer.word_index

In [61]:
def convert_text_to_index_array(text):
    # one really important thing that `text_to_word_sequence` does
    # is make all texts the same length -- in this case, the length
    # of the longest text in the set.
    return [dictionary[word] for word in kpt.text_to_word_sequence(text)]

In [62]:
allWordIndices = []
# for each tweet, change each token to its ID in the Tokenizer's word_index
for text in X_train:
    wordIndices = convert_text_to_index_array(text)
    allWordIndices.append(wordIndices)

In [63]:
# now we have a list of all tweets converted to index arrays.
# cast as an array for future usage.
allWordIndices = np.asarray(allWordIndices)

# create one-hot matrices out of the indexed tweets
X_train = tokenizer.sequences_to_matrix(allWordIndices, mode='binary')

In [64]:
y_train = keras.utils.to_categorical(y_train.factorize()[0], num_classes)
y_test = keras.utils.to_categorical(y_test.factorize()[0], num_classes)

In [65]:
print('y_train shape:', y_train.shape)
print('y_test shape:', y_test.shape)

y_train shape: (47689, 47690)
y_test shape: (11923, 47690)


## Training Models:

In [ ]:
print('Building model...')
model = Sequential()
model.add(Dense(512, input_shape=(max_words,)))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

history = model.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1)
score = model.evaluate(x_test, y_test,
                       batch_size=batch_size, verbose=1)
print('Test score:', score[0])
print('Test accuracy:', score[1])

Building model...
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 42920 samples, validate on 4769 samples
Epoch 1/5
22720/42920 [==============>...............] - ETA: 3:41 - loss: 10.8072 - acc: 0.0010

In [ ]:
# define the model
def define_model(length, vocab_size):
    # channel 1
    inputs1 = Input(shape=(length,))
    embedding1 = Embedding(vocab_size, 100)(inputs1)
    conv1 = Conv1D(filters=32, kernel_size=4, activation='relu')(embedding1)
    drop1 = Dropout(0.5)(conv1)
    pool1 = MaxPooling1D(pool_size=2)(drop1)
    flat1 = Flatten()(pool1)
    # channel 2
    inputs2 = Input(shape=(length,))
    embedding2 = Embedding(vocab_size, 100)(inputs2)
    conv2 = Conv1D(filters=32, kernel_size=6, activation='relu')(embedding2)
    drop2 = Dropout(0.5)(conv2)
    pool2 = MaxPooling1D(pool_size=2)(drop2)
    flat2 = Flatten()(pool2)
    # channel 3
    inputs3 = Input(shape=(length,))
    embedding3 = Embedding(vocab_size, 100)(inputs3)
    conv3 = Conv1D(filters=32, kernel_size=8, activation='relu')(embedding3)
    drop3 = Dropout(0.5)(conv3)
    pool3 = MaxPooling1D(pool_size=2)(drop3)
    flat3 = Flatten()(pool3)
    # merge
    merged = concatenate([flat1, flat2, flat3])
    # interpretation
    dense1 = Dense(10, activation='relu')(merged)
    outputs = Dense(1, activation='sigmoid')(dense1)
    model = Model(inputs=[inputs1, inputs2, inputs3], outputs=outputs)
    # compile
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    # summarize
    print(model.summary())
    plot_model(model, show_shapes=True, to_file='multichannel.png')
    return model

In [ ]:
# define model
model = define_model(max_words, vocab_size)
# fit model
model.fit([X_train, X_train, X_train], np.array(y_train), epochs=10, batch_size=16)